In [1]:
import pandas as pd

from typing import *

import yargy as y
import yargy.predicates as yp
import yargy.morph as ytm
import yargy.tokenizer as yt

from pymorphy2 import MorphAnalyzer
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd 

from pyhash import city_32
from rich import print, inspect
import joblib as jb
import json

from rich import print
import razdel

import os
from tqdm.auto import tqdm

from functools import lru_cache
CACHE_SIZE=10000
from src.contextual_morphology import UdMorphTokenizer, spacy_udpipe

In [2]:
tqdm.pandas()

/usr/local/lib/python3.8/dist-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
FILENAME = "results/results_youtube_emojipunct.pkl"
RST_CACHE_NAME = "cache/cache_w1.pkl"

In [4]:
model = spacy_udpipe.load_from_path(
    lang="ru",
    path="./data/models/russian-syntagrus-ud-2.5-191206.udpipe",
    meta={"description": "Custom 'hr' model"}
)
ud_tokenizer = UdMorphTokenizer(model)

In [5]:
def get_morph(word, cyr=True):
    morph = MorphAnalyzer()
    if cyr:
        return morph.lat2cyr(morph.parse(word)[0].tag)
    else:
        return morph.parse(word)[0].tag

In [6]:
rules = pd.read_csv("data/rules/rules_formatted.csv")

In [7]:
predicates_ = pd.read_csv("data/rules/predicates.csv")

In [8]:
types = set(predicates_.type.tolist())
if 'deverbal_noun' in types:
    deverbal_nouns = set(predicates_[predicates_.type == 'deverbal_noun']['predicate'].to_list())
else:
    deverbal_nouns = set()
    
if 'status_category' in types:
    status_categories = set(predicates_[predicates_.type == 'status_category']['predicate'].to_list())
else:
    status_categories = set()
    
predicates = set(predicates_[predicates_.type == 'predicate']['predicate'].to_list())

rule_specific = set(rules['predicate'].to_list())

In [9]:
all_predicates = predicates | rule_specific # | deverbal_nouns | status_categories |

In [10]:
all_predicates -= {'?'}

In [11]:
def create_predicate_rule(
    require_deverbal_noun: str,
    require_reflexive: str,
    require_status_category: str,
    predicate: str,
    predicate_type: str,
    **kwargs
):
    rule_id = f"predicate={predicate},deverbal={require_deverbal_noun},reflexive={require_reflexive},status_category={require_status_category},predicate_type={predicate_type}"
    return rule_id, y.rule(
        y.and_(
            req_predicate(predicate, predicate_type),
            req_deverbal(require_deverbal_noun),
            req_reflexive(require_reflexive)
        )
    )

In [12]:
def create_argument_role(argument_type: str, case: str, preposition: str, **kwargs):
    rule_id = f"argument_type={argument_type},case={case},preposition={preposition}"
    arg = y.and_(
        req_argument(),
        req_animacy(argument_type),
        req_case(case)
    )
    internal = y.or_(
        y.and_(
            yp.gram("ADJF"), 
            y.or_(
                yp.normalized("этот"),
                yp.normalized("тот")
            )
        ),
        y.not_(yp.gram("ADJF"))
    )
    
    rule = y.or_(
        y.rule(req_preposition(preposition), arg),
        y.rule(req_preposition(preposition), internal, arg)
    )
    return rule_id, rule

In [13]:
def req_deverbal(require_deverbal_noun: str = '?'):
    if require_deverbal_noun == '1': ## strictly deverbal noun
        return y.and_(
            yp.gram("NOUN"),
            yp.in_caseless(deverbal_nouns)
        )
    elif require_deverbal_noun == '0': ## strictly regular verb
        return y.or_(
            yp.gram("VERB"),
            #yp.gram("INFN") UD does not have infn
        )
    elif require_deverbal_noun == '?': ## anything
        return y.or_(
            y.and_(
                yp.gram("NOUN"),
                yp.in_caseless(deverbal_nouns)
            ),
            yp.gram("VERB"),
            #yp.gram("INFN") ud does not have infn
        )
    else:
        raise ValueError("Incorrect deverbal status")

In [14]:
def req_reflexive(reflexive_status: str = '?'):
    
    def is_reflexive_verb(verb: str):
        return verb.endswith("ся") or verb.endswith("сь")
    
    if reflexive_status == "1":
        return yp.custom(is_reflexive_verb)
    if reflexive_status == "0":
        return y.not_(yp.custom(is_reflexive_verb))
    elif reflexive_status == "?":
        return yp.true()
    else:
        raise ValueError ("Incorrect reflexive status")

In [15]:
def req_animacy(animacy: str = 'любой'):
    if animacy == 'любой':
        return yp.true()
    elif animacy == 'одуш.':
        return y.or_(
            y.not_(yp.gram('Inan')),
            yp.gram("Anim"),
            yp.gram("NOUN"),
            yp.gram("ADJ")
        )
    elif animacy == 'неодуш.':
        return y.or_(
            yp.gram('Inan'),
            y.not_(yp.gram("Anim")),
            #yp.gram("anim"),
            #yp.gram("NPRO"), considering pronouns as animacy now, todo - improve it
            yp.gram("ADJ")
        )
    else:
        raise ValueError("Incorrect Animacy Type")

In [16]:
def req_argument():
    return y.and_(
        y.not_(
            y.or_( ## prohibits arguments from being any of following parts-of-speech
                yp.gram('PART'),
                yp.gram("ADP"),
                yp.gram("CCONJ"),
                yp.gram('SCONJ'),
                yp.gram("INTJ"),
                yp.gram("ADJ"),
                yp.gram("VERB")
            )
        ),
        y.or_(
            yp.gram("NOUN"),
            yp.gram("PROPN"),
            yp.gram("PRON")
        )
    )

In [17]:
def req_predicate(word: str = "?", predicate_type: str = 'глаг'):
    # add predicate_type handling
    if predicate_type == 'глаг':
        predicate = y.or_(
            yp.gram("VERB"),
            #yp.gram("INFN")
        )
    elif predicate_type == 'сущ':
        predicate = y.or_(
            #yp.gram("INFN"),
            yp.gram("NOUN")
        )
    elif predicate_type == 'любой':
        predicate = y.or_(
            yp.gram("VERB"),
            #yp.gram("INFN"),
            yp.gram("NOUN")
        )
    else:
        raise ValueError("predicate_type must be глаг or сущ or любой")
    if word != '?':
        if "|" not in word:
            # single-word scope
            predicate = y.and_(
                yp.normalized(word),
                predicate
            )
        else:
            predicate_words = word.split("|")
            scope_rule = list(map(yp.normalized, predicate_words))
            scope_rule = y.or_(*scope_rule)
            predicate = y.and_(
                scope_rule,
                predicate
            )
        
    return predicate

In [18]:
from collections import defaultdict

In [19]:
def req_case(case: str = 'в'):
    mapping = {
        'в': 'Acc',
        'т': 'Abl',
        'д': 'Dat',
        'р': 'Gen',
        'и': 'Nom',
        'п': 'Loc'
    }
    
    if case not in mapping:
        raise ValueError(f"{case} unknown")
        
    case_rule = yp.gram(mapping[case])
    del mapping[case]
    not_rule = y.not_(y.or_(*(yp.gram(other_case) for other_case in mapping.values())))
    
    return y.and_(case_rule, not_rule)

In [20]:
def req_preposition(preposition: str = None):
    if preposition == 'None':
        return yp.true()
    else:
        return y.or_(
            y.and_(
                yp.gram("ADP"),
                yp.eq(preposition)
            )#,
            #y.not_(yp.gram("PREP"))
        )

In [21]:
def soft_parser_pass(parser, text):
    matches = []
    for match in parser.findall(text):
        matches.append({
            'text': " ".join([x.value for x in match.tokens]),
            'span': tuple(match.span)
        })

    return matches

In [22]:
def strict_parser_pass(parser, text):
    match = parser.match(text)
    if match is not None:
        matches = [{
            'text': " ".join([x.value for x in match.tokens]),
            'span': tuple(match.span)
        }]
    else:
        matches = []
    
    return matches

In [23]:
def create_rules(**kwargs):
    predicate_rule_id, predicate_rule = create_predicate_rule(**kwargs)
    argument_rule_id, argument_rule = create_argument_role(**kwargs)
    return {
        'predicate_id': predicate_rule_id,
        'argument_id': argument_rule_id,
        'predicate_parser': y.Parser(predicate_rule, tokenizer=ud_tokenizer),
        'argument_parser': y.Parser(argument_rule, tokenizer=ud_tokenizer)
    }

In [24]:
roleset = set(rules.role)

In [25]:
ruleset = {}

In [26]:
from tqdm.auto import tqdm

In [27]:
for role in roleset:
    ruleset[role] = []
    
    for rule_dict in tqdm(rules[rules.role == role].to_dict(orient='records'), desc=role):
        ruleset[role].append(create_rules(**rule_dict))

In [28]:
argument_rules = {}
for role in ruleset.keys():
    for rule in ruleset[role]:
        argument_rules[f"{rule['argument_id']}+{role}"] = {
            'role':role,
            'rule_id': rule['argument_id'],
            'argument_parser': rule['argument_parser']
        }

In [29]:
predicate_orient = {}
for role in ruleset.keys():
    for rule in ruleset[role]:
        rule_id = rule['predicate_id']
        if rule_id not in predicate_orient:
            predicate_orient[rule_id] = {}
            predicate_orient[rule_id]['predicate_parser'] = rule['predicate_parser']
            predicate_orient[rule_id]['arguments'] = []
            
        predicate_orient[rule_id]['arguments'].append({
            'role': role,
            'argument_id': rule['argument_id'],
            'argument_parser': rule['argument_parser']
        })

In [30]:
predicate_orient_rules = {}
for predicate_id in predicate_orient.keys():
    argument_tuples = set([
        f"{x['argument_id']}+{x['role']}" for x in predicate_orient[predicate_id]['arguments']
    ])
    predicate_orient_rules[predicate_id] = {
        'predicate_parser': predicate_orient[predicate_id]['predicate_parser'],
        'arguments': [argument_rules[key] for key in argument_tuples]
    }

In [31]:
import yargy.pipelines as pipelines

In [32]:
filter_pipeline = y.Parser(
    pipelines.morph_pipeline(list(all_predicates)),
    tokenizer=ud_tokenizer
)

In [33]:
def check_parseable(text, parser):
    return len(list(parser.findall(text))) > 0

In [34]:
class ArgumentExtractor:
    
    def __init__(self, *args, **kwargs):
        pass
    
    def extract(self, sentence: str) -> List[Dict[str, Any]]:
        pass

In [35]:
from ufal.udpipe import Model, Pipeline, ProcessingError
from predpatt import PredPatt, load_conllu
from predpatt.patt import Token
from predpatt import PredPattOpts
from predpatt.util.ud import dep_v1, dep_v2

class PredPattArgumentExtractor(ArgumentExtractor):
    def __init__(
        self,
        path_to_udpipe: str,
        resolve_relcl: bool = True,
        resolve_appos: bool = True,
        resolve_amod: bool = True,
        resolve_conj: bool = True,
        resolve_poss: bool = True,
        ud = dep_v2.VERSION
    ):
        super().__init__()
        self.model = Model.load(path_to_udpipe)
        self.pipeline = Pipeline(self.model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        self._error = ProcessingError()
        self._opts = PredPattOpts(
            resolve_relcl=resolve_relcl,
            resolve_appos=resolve_appos,
            resolve_amod=resolve_amod,
            resolve_conj=resolve_conj,
            resolve_poss=resolve_poss,
            ud=ud
        )
        
    @lru_cache(maxsize=100000)
    def extract(self, sentence: str) -> List[Dict[str, Any]]:
        processed = self.pipeline.process(sentence, self._error)
        if self._error.occurred():
            print(f"=== Error occurred: {self._error.message}")
            self._error = ProcessingError()
            return None
        else:
            conll_example = [ud_parse for sent_id, ud_parse in load_conllu(processed)][0]
            ppatt = PredPatt(conll_example, opts=self._opts)
            result = []
            for predicate in ppatt.instances:
                structure = {
                    'predicate': predicate.tokens,
                    'arguments': [x.tokens for x in predicate.arguments]
                }
                result.append(structure)
                
            return result        

In [36]:
from collections import defaultdict
class MainPhraseExtractor:
    
    def __init__(self, syntax_parser, pymorphy_analyzer):
        self.syntax = syntax_parser
        self.morph = pymorphy_analyzer

    def get_main_phrase(self, words, get_prep=False, verbose=False):
        markup = next(self.syntax.map([words]))
        forward = {}
        backward = defaultdict(list)
        token_map = {}
        candidates = []
        for token in markup.tokens:
            if token.head_id not in backward:
                backward[token.head_id] = []

            token_map[token.id] = token
            forward[token.id] = token.head_id
            backward[token.head_id].append(token.id)

            if token.id == token.head_id or token.head_id == '0':
                candidates.append(token.id)
             
        if verbose:
            print("forward ", forward)
            print("backward ", backward)
            print("candidates ", candidates)
                
        if len(candidates) == 0:
            return markup.tokens

        candidate = sorted(candidates, key=lambda x: len(backward[x]))[-1]
        if get_prep:
            prep_candidates = backward[candidate]
            prep_candidates = list(
                filter(lambda x: self.morph.tag(token_map[x].text)[0].POS == 'PREP', prep_candidates)
            )
            if len(prep_candidates) == 0:
                return [token_map[candidate]]
            
            prep = sorted(prep_candidates, key=lambda x: abs(int(x) - int(candidate)))[0]
            return (token_map[prep], token_map[candidate])

        return [token_map[candidate]]

In [37]:
class RstClauseSeparator:
    def __init__(self, udpipe=('tsa05.isa.ru', 3334), rst=('papertext.ru', 5555), cache_path="./rst-cache.pkl"):
        udpipe_host, udpipe_port = udpipe
        rst_host, rst_port = rst
        self.cache_path = cache_path
        self.ppl = PipelineCommon([
            (ProcessorRemote(udpipe_host, udpipe_port, '0'),
             ['text'],
             {'sentences': 'sentences',
              'tokens': 'tokens',
              'lemma': 'lemma',
              'syntax_dep_tree': 'syntax_dep_tree',
              'postag': 'ud_postag'}),
            (ProcessorMystem(delay_init=False),
             ['tokens', 'sentences'],
             {'postag': 'postag'}),
            (ConverterMystemToUd(),
             ['postag'],
             {'morph': 'morph',
              'postag': 'postag'}),
            (ProcessorRemote(rst_host, rst_port, 'default'),
             ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
             {'clauses': 'clauses'})])
        self.__cache = {}
        self.__hasher = city_32()
        if os.path.exists(self.cache_path):
            self.__cache = jb.load(self.cache_path)
        
    def extract(self, text):
        text_hash = self.__hasher(text)
        if text_hash in self.__cache:
            return self.__cache[text_hash]
        else:
            result = self.ppl(text)
            clauses = [x.text for x in result['clauses']]
            self.__cache[text_hash] = clauses
            return clauses
        
        
    def flush(self):
        jb.dump(self.__cache, self.cache_path)

In [38]:
class RoleLabeler:
    def __init__(
        self,
        argument_extractor: ArgumentExtractor,
        main_phrase_extractor: MainPhraseExtractor,
        filter_pipeline,
        predicate_ruleset,
        mode: str = 'soft',
        extend_arguments: bool = False
    ):
        
        self.argument_extractor = argument_extractor
        self.main_phrase_extractor = main_phrase_extractor
        self.filter_pipeline = filter_pipeline
        self.ruleset = predicate_ruleset
        self.extend_arguments = extend_arguments
        if mode == 'soft':
            self.pass_fn = soft_parser_pass
        elif mode == 'strict':
            self.pass_fn = strict_parser_pass
        else:
            raise ValueError(f"Incorrect mode = {mode}, can be 'soft' or 'strict'")
            
    def check_parse(self, text, parser):
        return len(self.pass_fn(parser, text)) > 0
    
    def run(self, sentence: str, return_applied_rules: bool = False):
        tokenized = map(lambda x: x.text, razdel.tokenize(sentence))
        words = list(map(lambda x: Token(x[0], x[1], None), enumerate(tokenized)))
        arg_groups = self.argument_extractor.extract(sentence)
        arg_groups = list(
            filter(
                lambda x: check_parseable(
                    " ".join([token.text for token in x['predicate']]),
                    self.filter_pipeline
                ),
                arg_groups
            )
        )
        result = []
        for group in arg_groups:
            
            predicate_txt = " ".join([token.text for token in group['predicate']])
            predicate_tokens = [token.text for token in group['predicate']]
            predicate_main = " ".join([x.text for x in self.main_phrase_extractor.get_main_phrase(predicate_tokens)])
            forward_map = {" ".join([token.text for token in argument]): argument for argument in group['arguments']}
            group_name = f"predicate={predicate_txt},arguments=[{','.join(forward_map.keys())}]"
            group_result = []
            
            # iterating over predicates to find match
            for predicate_id, predicate in self.ruleset.items():
                if self.check_parse(predicate_main, predicate['predicate_parser']):
                    predicate_result = {
                        'predicate': predicate_txt,
                        'predicate_analyzed': predicate_main,
                        'predicate_morph': get_morph(predicate_main),
                        'predicate_tokens': group['predicate'],
                        'arguments': []
                    }
                    if return_applied_rules:
                        predicate_result['applied_predicate_rule'] = predicate_id,

                    for argument in forward_map.keys():
                        argument_tokens = [x.text for x in forward_map[argument]]
                        offset = min(x.position for x in forward_map[argument]) - 1
                        argument_main_phrase = self.main_phrase_extractor.get_main_phrase(argument_tokens, True)
                        argument_main = " ".join([
                            x.text for x in argument_main_phrase
                        ])
                        argument_word = argument_main
                        
                        token_positions = [(offset + int(x.id)) for x in argument_main_phrase]
                        try:
                            if self.extend_arguments: # extending argument with up to 2 previous tokens to ensure preposition included
                                min_pos = min(token_positions)
                                if min_pos >= 2:
                                    argument_main = f"{words[min_pos - 2].text} {words[min_pos - 1].text} {argument_main}"
                                elif min_pos == 1:
                                    argument_main = f"{words[0].text} {argument_main}"
                        except IndexError as e:
                            print(f"Index error for {token_positions} at {words}")
                        
                        # iterating over possible arguments of matched predicate
                        roles = []
                        for argument_rule in predicate['arguments']:
                            parser = argument_rule['argument_parser']
                            rule_id = argument_rule['rule_id']
                            role = argument_rule['role']
                            if self.check_parse(argument_main, parser):
                                #print(f"Applied rule: {rule_id} to {argument_main}")
                                if return_applied_rules:
                                    roles.append({
                                        'role': role,
                                        'applied_rule': rule_id
                                    })
                                else:
                                    roles.append(role)
                                
                        if len(roles) > 0: 
                            predicate_result['arguments'].append({
                                'argument': argument,
                                'argument_main': argument_main,
                                'argument_morph': get_morph(argument_main),
                                'argument_analyzed': argument_word,
                                'argument_tokens': forward_map[argument],
                                'roles': tuple(roles)
                            })
                    if len(predicate_result['arguments']) > 0:
                        predicate_result['arguments'] = tuple(predicate_result['arguments'])
                        group_result.append(predicate_result)
            result.append({'group': group_name, 'parses': group_result})
        return result
            

In [39]:
class ConstraintEnforcer:
    def __init__(self, constraints=None):
        if constraints is None:
            constraints = list()
            
        self.constraints = constraints
        
    def add(self, constraint):
        self.constraints.append(constraint)
        
    def enforce(self, parse):
        a_parse = parse.copy()
        for constraint in self.constraints:
            a_parse = constraint(a_parse)
            if len(a_parse) == 0:
                return a_parse
            
        return a_parse

In [40]:
from navec import Navec
from slovnet import Syntax
navec = Navec.load('data/models/navec_news_v1_1B_250K_300d_100q.tar')
syntax = Syntax.load('data/models/slovnet_syntax_news_v1.tar')
_ = syntax.navec(navec)

In [41]:
main_phrase_extractor = MainPhraseExtractor(syntax, MorphAnalyzer())
extractor = PredPattArgumentExtractor("./data/models/russian-syntagrus-ud-2.5-191206.udpipe")
clause_extractor = RstClauseSeparator(cache_path="./experiments/rst-cache.pkl")

In [42]:
labeler = RoleLabeler(extractor, main_phrase_extractor, filter_pipeline, predicate_orient_rules, mode='soft', extend_arguments=True)

In [43]:
enforcer = ConstraintEnforcer()
def enforce_parseable_predicate(parse):
    if check_parseable(parse['predicate_analyzed'], filter_pipeline):
        return parse
    else:
        return {}
    
def reduce_duplicate_roles(parse):
    new_args = []
    for arg in parse['arguments']:
        arg['roles'] = tuple(set(arg['roles']))
        new_args.append(arg)
    parse['arguments'] = new_args
    return parse
    
def resolve_multiple_expirirencers(parse):
    if len(parse['arguments']) >= 2:
        parse_roles = set(arg['roles'] for arg in parse['arguments'])
        if ('экспериенцер',) in parse_roles:
            new_args = []
            for arg in parse['arguments']:
                if len(arg['roles']) >= 2:
                    new_roles = list(arg['roles'])
                    if 'экспериенцер' in new_roles:
                        new_roles.remove('экспериенцер')
                    arg['roles'] = tuple(new_roles)
                new_args.append(arg)
            parse['arguments'] = new_args
    return parse

def resolve_single_expiriencer(parse):
    parse_roles = [arg['roles'] for arg in parse['arguments'] if len(arg['roles']) >= 2]
    if len(parse_roles) > 0:
        n_exp = 0
        for role in parse_roles:
            if 'экспериенцер' in role:
                n_exp += 1
                
        if n_exp == 1:
            new_args = []
            for arg in parse['arguments']:
                if len(arg['roles']) >= 2 and 'экспериенцер' in arg['roles']:
                    arg['roles'] = ('экспериенцер',)
                new_args.append(arg)
            parse['arguments'] = new_args
    return parse
        
enforcer.add(enforce_parseable_predicate)
enforcer.add(reduce_duplicate_roles) # ('каузатор', 'экспериенцер', 'экспериенцер') ->  ('каузатор', 'экспериенцер')
enforcer.add(resolve_multiple_expirirencers)
enforcer.add(resolve_single_expiriencer) # ('каузатор', 'экспериенцер') -> ('экспериенцер')

In [44]:
pd.set_option("display.max_columns", 1000)

In [52]:
data = pd.read_parquet("./data/youtube/youtube_1yeat.parquet")

In [53]:
data.head()

,file,text
0,D5S_deYcRqI,У нас на всех ток шоу обсуждают всё кроме реал...
1,D5S_deYcRqI,Не волнуйся! Все будет хорошо!!\n
2,D5S_deYcRqI,А есле полезити занимать кредит у МВФ мерового...
3,D5S_deYcRqI,"@Александр Чемезов Я знаю и помню, когда не за..."
4,D5S_deYcRqI,Глупость! Умный истеблишмент США стравливает п...


In [50]:
import emoji
import re
from multiprocessing import Pool

In [48]:
def emoji_to_punct(target: str, symbol: str = ";"):
    return re.sub(
        r'{{(.*)}}'
        symbol,
        emoji.demojize(target, delimiters=("{{", "}}"))
    )

In [55]:
def parallel_apply(series, func, n_cores=4):
    pool = Pool(n_cores)
    series = pool.map(func, tqdm(series))
    pool.close()
    pool.join()
    return series

In [56]:
parallel_processed = parallel_apply(data['text'], emoji_to_punct, 12)

In [62]:
data['text'] = parallel_processed

In [63]:
data.shape

(596160, 2)

In [ ]:
parses = []
empty_parses = []
errors = []
subdata = data[:100000]
for row in tqdm(subdata.itertuples(), total=len(subdata)):
    text = row.text
    file = row.file
    
    sentences = [x.text for x in razdel.sentenize(text)]
    predicates = len(list(filter_pipeline.findall(text)))
    roles = []
    clauses = []
    n_tokens = 0
    for s in sentences:
        try:
            clauses += clause_extractor.extract(s.replace(". ", ""))
        except Exception as e:
            errors.append((s, e))
        n_tokens += len(list(razdel.tokenize(s)))
        
    for i, clause in enumerate(clauses):
        if check_parseable(clause, filter_pipeline):
            groups = labeler.run(clause)
            if len(groups) != 0:
                for j, group in enumerate(groups):
                    for k, parse in enumerate(group['parses']):
                        parse = enforcer.enforce(parse)
                        if len(parse) != 0:
                            parses.append({
                                'text': text,
                                'clause_text': clause,
                                'content_hash': file,
                                'clause_idx': i,
                                'group_idx': j,
                                'parse_idx': k,
                                'parse': parse
                            })
                        else:
                            empty_parses.append({
                                'text': text,
                                'clause_text': clause,
                                'content_hash': file,
                                'clause_idx': i,
                                'group_idx': j,
                                'parse_idx': k,
                                'parse': parse
                            })

In [64]:
parses

[{'text': '@Папина Дочка а зачем смотреть, что там увидит, он знает - дебила, зачем лишний раз расстраивать себя, хоть и не умного, но все же любимого.\n',
  'clause_text': '@Папина Дочка а зачем смотреть, что там увидит, он знает - дебила, зачем лишний раз расстраивать себя, хоть и не умного, но все же любимого.',
  'content_hash': 'XUOilPRZHDA',
  'clause_idx': 0,
  'group_idx': 0,
  'parse_idx': 0,
  'parse': {'predicate': 'зачем расстраивать',
   'predicate_analyzed': 'расстраивать',
   'predicate_morph': 'ИНФ,несов,перех',
   'predicate_tokens': [зачем/15, расстраивать/18],
   'arguments': [{'argument': 'лишний раз',
     'argument_main': 'дебила , лишний раз',
     'argument_morph': 'СУЩ,неод,жр мн,рд',
     'argument_analyzed': 'лишний раз',
     'argument_tokens': [лишний/16, раз/17],
     'roles': ('экспериенцер',)},
    {'argument': 'себя , хоть и не умного',
     'argument_main': 'лишний раз себя',
     'argument_morph': 'ДЕЕПР,сов,перех прош',
     'argument_analyzed': 'себ

In [79]:
len(parses)

69111

In [80]:
clause_extractor.flush()

In [82]:
from joblib import dump, load

In [83]:
dump({'full_parses': parses, 'empty_parses': empty_parses}, FILENAME)

['../results/results_youtube_emojipunct.pkl']

In [84]:
print("Done")